In [1]:
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DecimalType, TimestampType
from pyspark.sql.functions import *
import datetime
from settings import *
from decimal import *

In [2]:
""" Configuramos Spark """
conf = SparkConf()
conf.setAppName("ProcesamientoDatos")
conf.setMaster("local[*]")

In [3]:
spark = SparkSession.builder.config(conf=conf).getOrCreate()

Establecemos el esquema de la tabla, es decir, los nombres de las columnas debido a que los csv aportados no contienen cabeceras. Así facilitaremos las operaciones posteriormente, al poder acceder con el nombre a los datos. Las columnas serán las siguientes:

| NOMBRE             | SIGNIFICADO                                                |
|--------------------|------------------------------------------------------------|
| medallon           | Identificador md5sum del taxi                              |
| licencia           | Identificador md5sum de la licencia del taxi               |
| hora_subida        | Hora a la que se sube el/la/los/las pasajero/a/os/as       |
| hora_bajada        | Hora a la que se baja el/la/los/las pasajero/a/os/as       |
| duracion_viaje_seg | Duración del viaje en segundos                             |
| distancia_viaje    | Distancia del viaje en millas                              |
| longitud_subida    | Coordinada de longitud de la subida                        |
| latitud_subida     | Coordinada de latitud de la subida                         |
| longitud_bajada    | Coordinada de longitud de la bajada                        |
| latitud_bajada     | Coordinada de latitud de la bajada                         |
| tipo_pago          | Método de pago (Tarjeta o efectivo)                        |
| tarifa             | Cantidad de la tarifa en dólares                           |
| recargo            | Cantidad del recargo en dólares                            |
| tasas              | Cantidad de las tasas en dólares                           |
| propina            | Cantidad de la propina en dólares                          |
| peaje              | Cantidad de los peajes de los puentes o tuneles en dólares |
| cantidad_total     | Cantidad total en dólares                                  |

In [4]:
nombreColumnas = StructType([StructField("medallon", StringType(), True),
                             StructField("licencia", StringType(), True),
                             StructField("hora_subida", TimestampType(), True),
                             StructField("hora_bajada", TimestampType(), True),
                             StructField("duracion_viaje_seg", IntegerType(), True),
                             StructField("distancia_viaje", DecimalType(precision=10, scale=2), True),
                             StructField("longitud_subida", DecimalType(precision=18, scale=14), True),
                             StructField("latitud_subida", DecimalType(precision=18, scale=14), True),
                             StructField("longitud_bajada", DecimalType(precision=18, scale=14), True),
                             StructField("latitud_bajada", DecimalType(precision=18, scale=14), True),
                             StructField("tipo_pago", StringType(), True),
                             StructField("tarifa", DecimalType(precision=10, scale=2), True),
                             StructField("recargo", DecimalType(precision=10, scale=2), True),
                             StructField("tasas", DecimalType(precision=10, scale=2), True),
                             StructField("propina", DecimalType(precision=10, scale=2), True),
                             StructField("peaje", DecimalType(precision=10, scale=2), True),
                             StructField("cantidad_total", DecimalType(precision=10, scale=2), True)])

Leemos el csv estableciendo el esquema que queremos y el tipo de fecha establecida

In [19]:
data = spark.read.csv("./../datos/unprocessed/1quinientos.csv", schema=nombreColumnas, timestampFormat="yyyy-MM-dd HH:mm:ss")

In [20]:
data.show()

+--------------------+--------------------+--------------------+--------------------+------------------+---------------+------------------+-----------------+------------------+-----------------+---------+------+-------+-----+-------+-----+--------------+
|            medallon|            licencia|         hora_subida|         hora_bajada|duracion_viaje_seg|distancia_viaje|   longitud_subida|   latitud_subida|   longitud_bajada|   latitud_bajada|tipo_pago|tarifa|recargo|tasas|propina|peaje|cantidad_total|
+--------------------+--------------------+--------------------+--------------------+------------------+---------------+------------------+-----------------+------------------+-----------------+---------+------+-------+-----+-------+-----+--------------+
|5EE2C4D3BF57BDB45...|E96EF8F6E6122591F...|2013-01-01 00:00:...|2013-01-01 00:00:...|                26|           0.10|-73.99221000000000|40.72512400000000|-73.99164600000000|40.72665800000000|      CSH|  2.50|   0.50| 0.50|   0.00| 0

In [21]:
data.createOrReplaceTempView("unprocessed")

Primero deberiamos filtrar los datos para evitar errores en la transformación de las posiciones a cuadrículas y eliminar las lineas que sean invalidas. Estas se darán en varios casos:    

* Cuando el ID del taxi o de la licencia es nulo
* Cuando las horas de subida y bajada coinciden
* Cuando la duración del viaje sea igual a 0
* Cuando el tipo de pago es diferente a en efectivo o con tarjeta
* Cuando el precio total es igual a 0
* Cuando las posiciones de inicio y final son las mismas
* Cuando alguna coordenada de posicion no está dentro de los límites del grid

Recordamos que los limites del grid se encuentran en:

* Latitud inicial = 41.477182778
* Longitud inicial = -74.916578
* Latitud final = 40,129715978
* Longitud final = -73.120778

Por otro lado, hay datos que no nos aportan nada para las consultas que vamos a querer realizar, por tanto, no los tomaremos. Estos serán:

* distancia_viaje
* recargo
* tasas
* peaje

Primera consulta:

```sql
SELECT 
    medallon,
    licencia,
    hora_subida,
    hora_bajada,
    duracion_viaje_seg,
    longitud_subida,
    latitud_subida,
    longitud_bajada,
    latitud_bajada,
    tipo_pago,
    tarifa,
    propina,
    cantidad_total
FROM unprocessed 
WHERE   
    medallon <> '' AND 
    licencia <> '' AND
    hora_subida <> hora_bajada AND
    duracion_viaje_seg > 0 AND
    cantidad_total > 0 AND
    longitud_subida <> longitud_bajada AND
    latitud_subida <> latitud_bajada AND
    (tipo_pago == 'CHS' OR tipo_pago == 'CRD')
```

Segunda consulta

```sql
SELECT * FROM unprocessed
WHERE
    (longitud_subida >= -74.916578 AND longitud_subida <= -73.120778) AND
    (longitud_bajada >= -74.916578 AND longitud_bajada <= -73.120778) AND
    (latitud_subida >= 40,129715978 AND latitud_subida <= 41.477182778) AND 
    (latitud_bajada >= 40,129715978 AND latitud_bajada <= 41.477182778) AND
```

La comprobación de coordenadas la hago con las funciones de spark para evitar castings

In [22]:
data = spark.sql("SELECT medallon, licencia, hora_subida, hora_bajada, duracion_viaje_seg, longitud_subida, latitud_subida, longitud_bajada, latitud_bajada, tipo_pago, tarifa, propina, cantidad_total FROM unprocessed WHERE medallon <> '' AND licencia <> '' AND hora_subida <> hora_bajada AND duracion_viaje_seg > 0 AND cantidad_total > 0 AND longitud_subida <> longitud_bajada AND latitud_subida <> latitud_bajada AND (tipo_pago = 'CSH' OR tipo_pago = 'CRD')")

In [24]:
data = data.where(data.longitud_subida >= INITIAL_LONGITUDE) \
    .where(data.longitud_subida <= FINAL_LONGITUDE) \
    .where(data.longitud_bajada >= INITIAL_LONGITUDE) \
    .where(data.longitud_bajada <= FINAL_LONGITUDE) \
    .where(data.latitud_subida >= FINAL_LATITUDE) \
    .where(data.latitud_subida <= INITIAL_LATITUDE) \
    .where(data.latitud_bajada >= FINAL_LATITUDE) \
    .where(data.latitud_bajada <= INITIAL_LATITUDE)

In [25]:
data.show()

+--------------------+--------------------+--------------------+--------------------+------------------+------------------+-----------------+------------------+-----------------+---------+------+-------+--------------+
|            medallon|            licencia|         hora_subida|         hora_bajada|duracion_viaje_seg|   longitud_subida|   latitud_subida|   longitud_bajada|   latitud_bajada|tipo_pago|tarifa|propina|cantidad_total|
+--------------------+--------------------+--------------------+--------------------+------------------+------------------+-----------------+------------------+-----------------+---------+------+-------+--------------+
|5EE2C4D3BF57BDB45...|E96EF8F6E6122591F...|2013-01-01 00:00:...|2013-01-01 00:00:...|                26|-73.99221000000000|40.72512400000000|-73.99164600000000|40.72665800000000|      CSH|  2.50|   0.00|          3.50|
|CA6CD9BAED6A85E43...|77FFDF38272A60065...|2013-01-01 00:00:...|2013-01-01 00:01:...|                61|-73.97010000000000|4

In [26]:
data.count()

333971

Ahora lo que haremos será crear las nuevas columnas que tomando las posiciones de subida y bajada indiquen la cuadrícula de estas. Lo realizaremos con los siguientes cálculos:

* Cuadrícula de latitud = int(floor((INITIAL_LATITUDE - latitud_subida)/LATITUDE)) + 1
* Cuadrícula de longitud = int(floor(abs(INITIAL_LONGITUDE - longitud_dada)/LONGITUDE)) + 1

La consulta en el lenguaje sería similar a esta:

```sql
SELECT
    medallon,
    licencia,
    hora_subida,
    hora_bajada,
    duracion_viaje_seg,
    distancia_viaje,
    longitud_subida,
    latitud_subida,
    longitud_bajada,
    latitud_bajada,
    tipo_pago,
    tarifa,
    recargo,
    tasas,
    propina,
    peaje,
    cantidad_total,
    int(floor(abs(INITIAL_LONGITUDE - longitud_subida)/LONGITUDE)) as cuad_longitud_subida,
    int(floor((INITIAL_LATITUDE - latitud_subida)/LATITUDE)) as cuad_longitud_subida,
    int(floor(abs(INITIAL_LONGITUDE - longitud_bajada)/LONGITUDE)) as cuad_longitud_subida,
    int(floor((INITIAL_LATITUDE - latitud_bajada)/LATITUDE)) as cuad_longitud_subida,
FROM unprocessed
```

Función para calcular el día de la semana

In [27]:
def dia_fecha(fecha):
    return fecha.weekday()

calcular_dia = udf(dia_fecha, IntegerType())

In [28]:
data = data.withColumn("cuad_latitud_subida", floor((INITIAL_LATITUDE - data.latitud_subida)/LATITUDE) + 1) \
    .withColumn("cuad_longitud_subida", floor(abs(INITIAL_LONGITUDE - data.longitud_subida)/LONGITUDE) + 1) \
    .withColumn("cuad_latitud_bajada", floor((INITIAL_LATITUDE - data.latitud_bajada)/LATITUDE) + 1) \
    .withColumn("cuad_longitud_bajada", floor(abs(INITIAL_LONGITUDE - data.longitud_bajada)/LONGITUDE) + 1) \
    .withColumn("dia_semana", calcular_dia(data.hora_subida))

Aquí podemos apreciar el esquema final de los datos

In [29]:
data.printSchema()

root
 |-- medallon: string (nullable = true)
 |-- licencia: string (nullable = true)
 |-- hora_subida: timestamp (nullable = true)
 |-- hora_bajada: timestamp (nullable = true)
 |-- duracion_viaje_seg: integer (nullable = true)
 |-- longitud_subida: decimal(18,14) (nullable = true)
 |-- latitud_subida: decimal(18,14) (nullable = true)
 |-- longitud_bajada: decimal(18,14) (nullable = true)
 |-- latitud_bajada: decimal(18,14) (nullable = true)
 |-- tipo_pago: string (nullable = true)
 |-- tarifa: decimal(10,2) (nullable = true)
 |-- propina: decimal(10,2) (nullable = true)
 |-- cantidad_total: decimal(10,2) (nullable = true)
 |-- cuad_latitud_subida: long (nullable = true)
 |-- cuad_longitud_subida: long (nullable = true)
 |-- cuad_latitud_bajada: long (nullable = true)
 |-- cuad_longitud_bajada: long (nullable = true)
 |-- dia_semana: integer (nullable = true)



In [34]:
data.write.parquet("./../datos/processed/dios.parquet")

In [35]:
data.write.csv("./../datos/processed/dios.csv")